***STARTING TO TRAIN***

In [ ]:
import psycopg2
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from shap import TreeExplainer, Explainer
import pickle
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

In [ ]:
server = "localhost"
database = "db_02"
username = "postgres"
password = "ABcd@12#$"
source = "creditcard_train"

data_train=[]

try:
  conn = psycopg2.connect(dbname=database, user=username, password=password, host=server, port="5432", sslmode='disable')
except psycopg2.Error as e:
  print("Error connecting to PostgreSQL database:", e)
else:
  print("Connection established successfully!")

cursor = conn.cursor()

query =f"SELECT * FROM {source}"
cursor.execute(query)

for i in cursor.fetchall():
  data_train.append(i[1:])

column_names = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
data_train=pd.DataFrame(data_train, dtype=float, columns=column_names)
# print(data_train)

In [ ]:
# # Loading Train Data
# source="creditcard_test"

# data_train2 = pd.read_csv(f"{source}.csv")

# print(data_train2)

***PRE-PROCESSING***

In [ ]:
# Data Pre-Processing
data_train = data_train.fillna(...)

categorical_features = []

if len(categorical_features) > 0:
  encoder = LabelEncoder()
  for feature in categorical_features:
    data_train[feature] = encoder.fit_transform(data_train[feature])

scaler = StandardScaler()

features_to_scale = data_train.drop("Class", axis=1).columns
data_train_scaled = scaler.fit_transform(data_train[features_to_scale])

X_train = data_train_scaled
y_train = data_train["Class"]

***FORMATION OF MODEL AND EXPLAINER***

In [5]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
print("Accuracy:", accuracy)
confusion_mat = confusion_matrix(y_train, y_pred)
print("Confusion Matrix:\n", confusion_mat)
classification_rep = classification_report(y_train, y_pred)
print("Classification Report:\n", classification_rep)
roc_auc = roc_auc_score(y_train, y_pred)
print("AUC-ROC:", roc_auc)

Accuracy: 1.0
Confusion Matrix:
 [[213207      0]
 [     0    398]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    213207
         1.0       1.00      1.00      1.00       398

    accuracy                           1.00    213605
   macro avg       1.00      1.00      1.00    213605
weighted avg       1.00      1.00      1.00    213605

AUC-ROC: 1.0


In [6]:
explainer = Explainer(model, X_train)
shap_values = explainer(X_train)

with open("model_rf.pkl", "wb") as f:
    pickle.dump(model, f)

with open("explainer_rf.pkl", "wb") as f:
    pickle.dump(explainer, f)

100%|===================| 427180/427210 [312:34<00:01]         

ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 0.010000, while the model output was 0.000000. If this difference is acceptable you can set check_additivity=False to disable this check.